In [1]:
import asyncio
from playwright.async_api import async_playwright
import qrcode
from PIL import Image
import io
import base64
from pyzbar import pyzbar
import os

In [3]:
playwright = await async_playwright().start()
browser = await playwright.chromium.launch(headless=False)
context = await browser.new_context()
page = await context.new_page()

In [4]:
await page.goto("https://web.whatsapp.com")

<Response url='https://web.whatsapp.com/' request=<Request url='https://web.whatsapp.com/' method='GET'>>

In [34]:
async def extract_message_details(message):
    name_element = await message.query_selector("div._aou8._aj_h > span")
    name = await name_element.inner_text() if name_element else "N/A"

    time_element = await message.query_selector("div._ak8i")
    time = await time_element.inner_text() if time_element else "N/A"

    message_element = await message.query_selector("div._ak8k > span > span")
    message = await message_element.inner_text() if message_element else "N/A"

    return {"name": name, "time": time, "message": message}

In [35]:
async def fetch_latest_message():
    selector = 'div[role="listitem"]'
    # or selector = '//*[@id="pane-side"]/div[2]/div/div/child::div'

    await page.wait_for_selector(selector, timeout=1000)
    list_items = await page.query_selector_all(selector)

    for item in list_items:
        transform = await item.evaluate(
            "element => window.getComputedStyle(element).transform"
        )

        if transform:
            if "matrix" in transform:
                parts = transform.replace("matrix(", "").replace(")", "").split(", ")
                if len(parts) == 6:
                    translate_y = float(parts[5])
                    if translate_y == 0:
                        return await extract_message_details(item)
            elif "translateY(0px)" in transform:
                print("Div with role='listitem' and translateY=72px found.")
                return await extract_message_details(item)

    print("Desired div not found.")
    return None


await fetch_latest_message()

{'name': 'Kausthubh', 'time': '12:24 AM', 'message': 'fuck'}

In [48]:
async def send_message(name: str, message: str):
    search_input_selector = (
        "div[contenteditable='true'][role='textbox'][data-lexical-editor='true']"
    )
    await page.wait_for_selector(search_input_selector, timeout=60000)
    search_input = await page.query_selector(search_input_selector)

    if search_input:
        await search_input.click()

        # await page.wait_for_timeout(2000)

        await page.keyboard.down("Control")
        await page.keyboard.press("A")
        await page.keyboard.up("Control")
        await page.keyboard.press("Backspace")

        # await page.wait_for_timeout(1000)

        await search_input.type(name, delay=20)
        await page.keyboard.press("Enter")

        # await page.wait_for_timeout(2000)

        await page.keyboard.down("Control")
        await page.keyboard.press("A")
        await page.keyboard.up("Control")
        await page.keyboard.press("Backspace")

        # await page.wait_for_timeout(1000)
        await page.keyboard.type(message, delay=20)
        await page.keyboard.press("Enter")
        await page.wait_for_timeout(100)
        await page.keyboard.press("Enter")

        await page.wait_for_timeout(200)
    else:
        print("Search input not found.")

In [52]:
async def watch_new_messages():
    """
    Continuously watches for new messages and yields them as they arrive.
    """
    new_message = await fetch_latest_message()
    assert new_message is not None

    name = new_message.get("name", None)
    message = new_message.get("message", None)
    time = new_message.get("time", None)

    await asyncio.sleep(1)

    while True:
        try:
            new_message = await fetch_latest_message()
            if new_message:
                if (
                    new_message.get("name", None) != name
                    or new_message.get("time", None) != time
                ):
                    name = new_message.get("name", None)
                    message = new_message.get("message", None)
                    time = new_message.get("time", None)
                    yield new_message
        except Exception as e:
            print(f"Error while fetching latest message: {e}")
        await asyncio.sleep(1)


async for message in watch_new_messages():
    print("New message received:")
    print(message)
    await send_message(message["name"], "heyy! what's up?")

New message received:
{'name': 'Kausthubh', 'time': '2:05 AM', 'message': 'boy'}


CancelledError: 

In [67]:
while True:
    m = await get_list_of_messages()
    print(m[0]["sender"])

    if m[0]["unread"] == True:
        await search_person(
            m[0]["sender"],
            "This is a test message for my hackathon project please ignore",
        )
        break
    else:
        print("No unread messages.")

Ryan Gosling
No unread messages.
Ryan Gosling
No unread messages.
Ryan Gosling
No unread messages.
Ryan Gosling
No unread messages.
Ryan Gosling
No unread messages.
Ryan Gosling
No unread messages.
Ryan Gosling
No unread messages.
Ryan Gosling
No unread messages.
Ryan Gosling
No unread messages.
Ryan Gosling
No unread messages.
Ryan Gosling
No unread messages.
Ryan Gosling
No unread messages.
Ryan Gosling
No unread messages.
Ryan Gosling
No unread messages.
Ryan Gosling
No unread messages.
Ryan Gosling
No unread messages.
Ryan Gosling
No unread messages.
Ryan Gosling
No unread messages.
Ryan Gosling
No unread messages.
Ryan Gosling
No unread messages.
Ryan Gosling
No unread messages.
Ryan Gosling
No unread messages.
Ryan Gosling
No unread messages.
Ryan Gosling
No unread messages.
Ryan Gosling
No unread messages.
Ryan Gosling
No unread messages.
Ryan Gosling
No unread messages.
Ryan Gosling
No unread messages.
Ryan Gosling
No unread messages.
Ryan Gosling
No unread messages.
Ryan Gosli

CancelledError: 

In [ ]:
def decode_qr_from_image(image_bytes):
    try:
        image = Image.open(io.BytesIO(image_bytes))
        decoded_objects = pyzbar.decode(image)
        for obj in decoded_objects:
            return obj.data.decode("utf-8")
        return None
    except Exception as e:
        print(f"Failed to decode QR from image: {e}")
        return None


def display_qr_in_terminal(data):
    try:
        qr = qrcode.QRCode()
        qr.add_data(data)
        qr.make()
        qr.print_ascii(invert=True)
    except Exception as e:
        print(f"Failed to display QR in terminal: {e}")

In [ ]:
qrinfo = None

try:
    await page.wait_for_selector("div._akau", timeout=600)
    print("QR code container detected.")

    qr_div = await page.query_selector("div._akau")
    if qr_div:
        qr_info = await qr_div.get_attribute("data-ref")
        if qr_info:
            qrinfo = qr_info
    else:
        print("data-ref attribute not found. Proceeding to screenshot method.")
except Exception as e:
    print(f"Error while extracting data-ref: {e}")

In [ ]:
qrinfo

In [ ]:
display_qr_in_terminal(qrinfo)

In [ ]:
screenshot = await page.screenshot()
qr_info = decode_qr_from_image(screenshot)
if qr_info:
    return qr_info
else:
    print("Failed to decode QR code from screenshot.")
    return None

In [ ]:
async def extract_login_qr(self):
    assert page != None

In [ ]:
try:
    # Wait for the search input to be visible using a more stable selector
    search_input_selector = (
        "div[contenteditable='true'][role='textbox'][data-lexical-editor='true']"
    )
    await page.wait_for_selector(search_input_selector, timeout=60000)
    search_input = await page.query_selector(search_input_selector)

    if search_input:
        # Type the name 'kausthubh' into the search input
        await search_input.click()
        await search_input.type(
            "kausthubh", delay=100
        )  # Adding a slight delay can help with typing simulation

        # Wait for search results to load
        await page.wait_for_timeout(2000)

        # Wait for the specific search result to be visible and click on it
        search_result_selector = "p.selectable-text[title='kausthubh']"
        search_result = await page.wait_for_selector(
            search_result_selector, timeout=60000
        )

        if search_result:
            await search_result.click()
            print("Chat with 'kausthubh' opened.")
        else:
            print(
                "Search result for 'kausthubh' not found. Clicking on the first chat."
            )

            # Selector for the first chat in the chat list
            first_chat_selector = (
                "div.x1y332i5:nth-child(3)"  # Update this selector based on actual DOM
            )
            first_chat = await page.wait_for_selector(
                first_chat_selector, timeout=60000
            )

            if first_chat:
                await first_chat.click()
                print("First chat clicked.")
            else:
                print("Failed to find the first chat.")
    else:
        print("Search input not found.")
except Exception as e:
    print(f"Error while searching for 'kausthubh': {e}")

## Closing the browser and saving the context

In [ ]:
await context.storage_state(path="browser_context.json")

if browser:
    await browser.close()
elif context:
    await context.close()

await playwright.stop()

In [ ]:
page2 = await browser.new_page()